In [1]:
!nvidia-smi

Fri Oct 29 18:36:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:8B:00.0 Off |                    0 |
| N/A   34C    P0    36W / 300W |   1385MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
DATA_DIR = '../data/'

In [2]:
import pandas as pd
import numpy as np

In [3]:
!ls ../data

chronom_test.csv   gas_test.csv     plavki_train.csv	   sip_train.csv
chronom_train.csv  gas_train.csv    produv_test.csv	   target_train.csv
chugun_test.csv    lom_test.csv     produv_train.csv
chugun_train.csv   lom_train.csv    sample_submission.csv
datka.zip	   plavki_test.csv  sip_test.csv


In [4]:
df_train_static = pd.read_csv(DATA_DIR + 'plavki_train.csv').drop_duplicates('NPLV').reset_index(drop=True)
df_test_static = pd.read_csv(DATA_DIR + 'plavki_test.csv')

target = pd.read_csv(DATA_DIR + 'target_train.csv')
df_train_static = df_train_static.merge(target, on='NPLV', how='left')

for plavki in [df_train_static, df_test_static]:
    plavki['plavka_VR_NACH'] = pd.to_datetime(plavki['plavka_VR_NACH'])
    plavki['plavka_VR_KON'] = pd.to_datetime(plavki['plavka_VR_KON'])
    plavki['plav_seconds'] = (((plavki.plavka_VR_KON.values - plavki.plavka_VR_NACH.values))*1e-9).astype(int)
df_train_static.head()

,NPLV,plavka_VR_NACH,plavka_VR_KON,plavka_NMZ,plavka_NAPR_ZAD,plavka_STFUT,plavka_TIPE_FUR,plavka_ST_FURM,plavka_TIPE_GOL,plavka_ST_GOL,TST,C,plav_seconds
0,510008,2021-01-01 03:08:11,2021-01-01 03:51:10,С255,МНЛЗ,971,цилиндрическая,11,5 сопловая,11,1690,0.060,2579
1,510009,2021-01-01 04:00:44,2021-01-01 05:07:28,С255,МНЛЗ,972,цилиндрическая,12,5 сопловая,12,1683,0.097,4004
2,510010,2021-01-01 05:12:29,2021-01-01 06:00:53,Ст3пс/Э,Изл,973,цилиндрическая,13,5 сопловая,13,1662,0.091,2904
3,510011,2021-01-01 06:13:48,2021-01-01 07:08:39,Св-08А.z02,Изл,974,цилиндрическая,14,5 сопловая,14,1609,0.410,3291
4,510012,2021-01-01 07:13:44,2021-01-01 08:01:59,SC2M/ЭТ,МНЛС,975,цилиндрическая,15,5 сопловая,15,1682,0.120,2895


In [6]:
static_cat_feats = ['plavka_NAPR_ZAD', 'plavka_TIPE_FUR', 'plavka_TIPE_GOL', 'plavka_NMZ']
static_num_feats = ['plavka_STFUT','plavka_ST_FURM','plav_seconds','plavka_ST_GOL']
target_fts = ['TST','C']
ts_cat_feats = ['NMSYP']
ts_num_feats = ['V', 'T', 'O2', 'N2', 'H2', 'CO2', 'CO', 'AR',
       'T фурмы 1', 'T фурмы 2', 'O2_pressure', 'RAS', 'POL', 'VDSYP',
       'VSSYP', 'VES_ch', 'T_ch', 'SI_ch', 'MN_ch', 'S_ch', 'P_ch', 'CR_ch',
       'NI_ch', 'CU_ch', 'V_ch', 'TI_ch']

In [ ]:
def make_dataset(data_dir, postfix = 'train'):
    plavki = pd.read_csv(data_dir + 'plavki_'+postfix + '.csv')
    plavki['plavka_VR_NACH'] = pd.to_datetime(plavki['plavka_VR_NACH'])
    plavki['plavka_VR_KON'] = pd.to_datetime(plavki['plavka_VR_KON'])
    gas = pd.read_csv(data_dir + 'gas_'+postfix + '.csv')
    gas['Time'] = pd.to_datetime(gas['Time'])
    produv = pd.read_csv(data_dir + 'produv_'+postfix + '.csv')
    produv['SEC'] = pd.to_datetime(produv['SEC'])
    sip = pd.read_csv(data_dir + 'sip_'+postfix + '.csv')
    sip['DAT_OTD'] = pd.to_datetime(sip['DAT_OTD'])
    chugun = pd.read_csv(data_dir + 'chugun_'+postfix + '.csv')
    chugun.columns = ['NPLV'] + [x+'_ch' for x in chugun.columns.tolist()[1:-1]] + ['DATA_ZAMERA']
    
    dfs = []
    for g,df in gas.groupby('NPLV'):
        print(g)
        p = plavki.loc[plavki.NPLV == g]
        p['plav_seconds'] = int(((p.plavka_VR_KON.values[0] - p.plavka_VR_NACH.values[0])).item()*1e-9)
        df1 = df.loc[df.Time <= p.plavka_VR_KON.values[0]]
        ppdf = produv.loc[(produv.NPLV == g) & (produv.SEC <= p.plavka_VR_KON.values[0])].drop(columns=['NPLV'])
        df1 = pd.merge_asof(df1, ppdf, left_on='Time', right_on='SEC').drop(columns=['SEC'])
        psip = sip.loc[(produv.NPLV == g) & (sip.DAT_OTD <= p.plavka_VR_KON.values[0])].drop(columns=['NPLV'])
        df1 = pd.merge_asof(df1, psip, left_on='Time', right_on='DAT_OTD').drop(columns=['DAT_OTD'])
        df1 = df1.merge(chugun, on='NPLV', how='left')
        df1 = df1.drop(columns = ['DATA_ZAMERA'])
        dfs.append(df1)
        #if len(dfs) > 100:
        #    break
        
    df = pd.concat(dfs, ignore_index = True).reset_index(drop=True)
    df[ts_num_feats] = df[ts_num_feats].interpolate()
    return df

df_train = make_dataset(DATA_DIR, 'train')
df_test = make_dataset(DATA_DIR, 'test')

510008


/tmp/ipykernel_287456/1568610564.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p['plav_seconds'] = int(((p.plavka_VR_KON.values[0] - p.plavka_VR_NACH.values[0])).item()*1e-9)


510009
510010
510011
510012
510013
510014
510015
510016
510017
510018
510019
510020
510021
510022
510023
510024
510026
510027
510028
510030
510031
510032
510033
510037
510040
510043
510044
510045
510047
510048
510049
510050
510051
510052
510053
510055
510057
510058
510059
510060
510061
510062
510063
510064
510066
510067
510068
510069
510070
510071
510072
510073
510074
510075
510076
510077
510078
510079
510080
510081
510082
510083
510084
510087
510088
510089
510090
510091
510092
510093
510094
510095
510096
510097
510098
510099
510100
510101
510102
510103
510104
510105
510106
510107
510108
510109
510110
510111
510112
510114
510115
510116
510117
510118
510119
510120
510121
510122
510124
510125
510126
510128
510130
510131
510132
510133
510134
510135
510136
510137
510138
510139
510140
510141
510142
510143
510144
510145
510146
510147
510148
510149
510151
510152
510153
510156
510157
510158
510159
510160
510161
510163
510164
510165
510166
510167
510168
510169
510170
510171
510172
510174
510175

511321
511322
511323
511324
511325
511326
511327
511328
511329
511330
511331
511332
511333
511334
511335
511336


In [6]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

le = LabelEncoder()
le.fit(df_train_static['plavka_NAPR_ZAD'])
df_train_static['plavka_NAPR_ZAD'] = le.transform(df_train_static['plavka_NAPR_ZAD'])
df_test_static['plavka_NAPR_ZAD'] = le.transform(df_test_static['plavka_NAPR_ZAD'])

le = LabelEncoder()
le.fit(df_train_static['plavka_TIPE_FUR'])
df_train_static['plavka_TIPE_FUR'] = le.transform(df_train_static['plavka_TIPE_FUR'])
df_test_static['plavka_TIPE_FUR'] = le.transform(df_test_static['plavka_TIPE_FUR'])

le = LabelEncoder()
plavka_nmz = set(df_train_static['plavka_NMZ'].unique().tolist()) & set(df_test_static['plavka_NMZ'].unique().tolist())
df_train_static.loc[~df_train_static['plavka_NMZ'].isin(plavka_nmz), 'plavka_NMZ'] = 'None'
df_test_static.loc[~df_test_static['plavka_NMZ'].isin(plavka_nmz), 'plavka_NMZ'] = 'None'
le.fit(df_train_static['plavka_NMZ'])
df_train_static['plavka_NMZ'] = le.transform(df_train_static['plavka_NMZ'])
df_test_static['plavka_NMZ'] = le.transform(df_test_static['plavka_NMZ'])

le = LabelEncoder()
plavka_nmz = set(df_train_static['plavka_TIPE_GOL'].unique().tolist()) & set(df_test_static['plavka_TIPE_GOL'].unique().tolist())
df_train_static.loc[~df_train_static['plavka_TIPE_GOL'].isin(plavka_nmz), 'plavka_TIPE_GOL'] = 'None'
df_test_static.loc[~df_test_static['plavka_TIPE_GOL'].isin(plavka_nmz), 'plavka_TIPE_GOL'] = 'None'
le.fit(df_train_static['plavka_TIPE_GOL'])
df_train_static['plavka_TIPE_GOL'] = le.transform(df_train_static['plavka_TIPE_GOL'])
df_test_static['plavka_TIPE_GOL'] = le.transform(df_test_static['plavka_TIPE_GOL'])

ss = StandardScaler()
ss.fit(df_train_static[static_num_feats])
df_train_static[static_num_feats] = ss.transform(df_train_static[static_num_feats])
df_test_static[static_num_feats] = ss.transform(df_test_static[static_num_feats])

target_scaler = StandardScaler()
df_train_static[target_fts] = target_scaler.fit_transform(df_train_static[target_fts])

le = LabelEncoder()
plavka_nmz = set(df_train['NMSYP'].unique().tolist()) & set(df_test['NMSYP'].unique().tolist())
df_train.loc[~df_train['NMSYP'].isin(plavka_nmz), 'NMSYP'] = 'None'
df_test.loc[~df_test['NMSYP'].isin(plavka_nmz), 'NMSYP'] = 'None'
le.fit(df_train['NMSYP'])
df_train['NMSYP'] = le.transform(df_train['NMSYP'])
df_test['NMSYP'] = le.transform(df_test['NMSYP'])

ss = StandardScaler()
ss.fit(df_train[ts_num_feats])
df_train[ts_num_feats] = ss.transform(df_train[ts_num_feats])
df_test[ts_num_feats] = ss.transform(df_test[ts_num_feats])

In [7]:
df_train

,NPLV,Time,V,T,O2,N2,H2,CO2,CO,AR,...,T_ch,SI_ch,MN_ch,S_ch,P_ch,CR_ch,NI_ch,CU_ch,V_ch,TI_ch
0,510008,2021-01-01 03:08:11.437,-0.648392,-0.527127,0.937859,0.793358,-0.283170,-1.196558,-0.706850,0.420513,...,-0.444865,-0.545556,-0.074593,1.713952,0.842314,1.711605,0.147078,0.803317,1.101082,-0.090980
1,510008,2021-01-01 03:08:12.437,-0.648392,-0.527127,0.938872,0.793675,-0.282793,-1.198191,-0.706894,0.419008,...,-0.444865,-0.545556,-0.074593,1.713952,0.842314,1.711605,0.147078,0.803317,1.101082,-0.090980
2,510008,2021-01-01 03:08:13.437,-0.635787,-0.529519,0.939886,0.793991,-0.282417,-1.199824,-0.706938,0.417503,...,-0.444865,-0.545556,-0.074593,1.713952,0.842314,1.711605,0.147078,0.803317,1.101082,-0.090980
3,510008,2021-01-01 03:08:14.437,-0.623184,-0.530714,0.940900,0.794308,-0.282041,-1.201457,-0.706983,0.415998,...,-0.444865,-0.545556,-0.074593,1.713952,0.842314,1.711605,0.147078,0.803317,1.101082,-0.090980
4,510008,2021-01-01 03:08:15.437,-0.635787,-0.534302,0.941913,0.794624,-0.281664,-1.203090,-0.707027,0.414493,...,-0.444865,-0.545556,-0.074593,1.713952,0.842314,1.711605,0.147078,0.803317,1.101082,-0.090980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315771,510125,2021-01-06 10:30:16.437,-0.825530,-0.951664,-0.185954,0.854826,-0.366402,-0.111855,-0.658951,1.168612,...,1.190704,1.508932,0.829609,0.353236,1.141905,0.170773,0.147078,1.869034,0.716310,1.193907
315772,510125,2021-01-06 10:30:17.437,-0.800152,-0.951664,-0.183264,0.854686,-0.366326,-0.111855,-0.659067,1.168248,...,1.190704,1.508932,0.829609,0.353236,1.141905,0.170773,0.147078,1.869034,0.716310,1.193907
315773,510125,2021-01-06 10:30:18.437,-0.800152,-0.951664,-0.180574,0.854546,-0.366249,-0.111855,-0.659183,1.167884,...,1.190704,1.508932,0.829609,0.353236,1.141905,0.170773,0.147078,1.869034,0.716310,1.193907
315774,510125,2021-01-06 10:30:19.437,-0.762127,-0.951664,-0.177883,0.854406,-0.366172,-0.111855,-0.659298,1.167520,...,1.190704,1.508932,0.829609,0.353236,1.141905,0.170773,0.147078,1.869034,0.716310,1.193907


In [62]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

class EvrazDatasetV(Dataset):
    def __init__(self, feat_df, static_train_df):
        self.feat_df = feat_df.copy()
        self.static_train_df = static_train_df.copy()
        
        self.nlmv =  self.feat_df.NPLV.unique()

    def __len__(self):
        return len(self.nlmv)

    def __getitem__(self, idx):    
        nlmv = self.nlmv[idx]
        return {"static_cat": self.static_train_df.loc[self.static_train_df.NPLV == nlmv, static_cat_feats].values[0],
                "static_numeric": self.static_train_df.loc[self.static_train_df.NPLV == nlmv, static_num_feats].values[0],
                "ts_cat": self.feat_df.loc[self.feat_df.NPLV == nlmv, ts_cat_feats].values,
                "ts_numeric": self.feat_df.loc[self.feat_df.NPLV == nlmv, ts_num_feats].fillna(0).values,
                "y": self.static_train_df.loc[self.static_train_df.NPLV == nlmv, target_fts].values[0]
               }

In [63]:
class AttentionWeightedAverage(nn.Module):
    def __init__(self, hidden_dim: int, return_attention: bool = False):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.return_attention = return_attention
        
        self.attention_vector = nn.Parameter(
            torch.empty(self.hidden_dim, dtype=torch.float32),
            requires_grad=True,
        )
        nn.init.xavier_normal_(self.attention_vector.unsqueeze(-1))

    def forward(
        self, x: torch.Tensor, mask: torch.Tensor
    ):
        logits = x.matmul(self.attention_vector)
        ai = (logits - logits.max()).exp()

        ai = ai * mask
        att_weights = ai / (ai.sum(dim=1, keepdim=True) + 1e-12)
        weighted_input = x * att_weights.unsqueeze(-1)
        output = weighted_input.sum(dim=1)

        if self.return_attention:
            return output, att_weights
        else:
            return output, None
        

CONV_FEAT_SIZE = 8
CONV_DELS = [1,2,4,8,16,32,64]

class EvrazModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.embs0 = nn.Embedding(3, 3)
        self.embs1 = nn.Embedding(2, 3)
        self.embs2 = nn.Embedding(4, 3)
        self.embs3 = nn.Embedding(29, 3)
        self.embs4 = nn.Embedding(4, 3)
        
        self.convs = nn.ModuleList([
            nn.Sequential(
                torch.nn.Conv1d(self.get_conv_size(i), CONV_FEAT_SIZE, 2, stride=1, 
                                padding=0, dilation=d, bias=False),
                nn.BatchNorm1d(CONV_FEAT_SIZE),
                nn.SiLU()
            )
            for i,d in enumerate(CONV_DELS)
        ])
        self.conv_attn = AttentionWeightedAverage(CONV_FEAT_SIZE)
                
        self.head = nn.Sequential(
            nn.LayerNorm(CONV_FEAT_SIZE+len(static_num_feats)+3*4),
            nn.Linear(CONV_FEAT_SIZE+len(static_num_feats)+3*4, 8),
            nn.LayerNorm(8),
            nn.SiLU(),
            nn.Linear(8, 8),
            nn.LayerNorm(8),
            nn.SiLU(),
            nn.Linear(8,2)
        )
        
    def get_conv_size(self, i):
        if i == 0:
            return len(ts_num_feats)+3
        return CONV_FEAT_SIZE
    
    def forward(self, static_cat, static_numeric, ts_cat, ts_numeric):
        x0 = self.embs0(static_cat[:,0])
        x1 = self.embs1(static_cat[:,1])
        x2 = self.embs2(static_cat[:,2])
        x3 = self.embs3(static_cat[:,3])
        x4 = static_numeric
        
        y0 = self.embs4(ts_cat[:,:,0])
        y = torch.cat([y0,ts_numeric], dim=-1).transpose(1,2)
        
        for m in self.convs:
            y = m(y)
        
        y = y.transpose(1,2)
        y = self.conv_attn(y, torch.ones((y.shape[0], y.shape[1])).to(y.device))[0]
        
        x = torch.cat([y,x0,x1,x2,x3,x4], dim=-1)
        
        return self.head(x)

In [65]:
model = EvrazModel().to(device)

In [66]:
ds = EvrazDatasetV(df_train, df_train_static)
dl = DataLoader(ds, shuffle=False, batch_size=1, num_workers=0)

In [67]:
for x in dl:
    out = model(x["static_cat"].long().to(device),
          x['static_numeric'].float().to(device),
          x['ts_cat'].long().to(device),
          x['ts_numeric'].float().to(device)
         )
    break

In [68]:
out

tensor([[-0.2516,  0.0375]], grad_fn=<AddmmBackward0>)

In [ ]:
def metric(answers, user_csv):
    delta_c = np.abs(np.array(answers[:,1]) - np.array(user_csv[:,1]))
    hit_rate_c = np.int64(delta_c < 0.02)

    delta_t = np.abs(np.array(answers[:,0]) - np.array(user_csv[:,0]))
    hit_rate_t = np.int64(delta_t < 20)

    N = np.size(answers[:,0])

    return np.sum(hit_rate_c + hit_rate_t) / 2 / N

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=239)

nplvs = df_train_static.NPLV.unique()
for vipl_train, vipl_valid in kf.split(nplvs):
    break
nplv_train = [nplvs[x] for x in vipl_train]
nplv_valid = [nplvs[x] for x in vipl_valid]

In [ ]:
train_ds = EvrazDatasetV(df_train.loc[df_train.NPLV.isin(nplv_train)].reset_index(drop=True), 
                         df_train_static.loc[df_train_staticNPLV.isin(nplv_train)].reset_index(drop=True))
train_dl = DataLoader(train_ds, shuffle=True, batch_size=1, num_workers=0)
valid_ds = EvrazDatasetV(df_train.loc[~df_train.NPLV.isin(nplv_train)].reset_index(drop=True), 
                         df_train_static.loc[~df_train_staticNPLV.isin(nplv_train)].reset_index(drop=True))
valid_dl = DataLoader(valid_ds, shuffle=False, batch_size=1, num_workers=0)

criterion = nn.L1Loss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=0.01)

model.train();
optimizer.zero_grad()

best_metric = 0
for epoch in range(100):
    losses = []
    for x in train_dl:
        out = model(x["static_cat"].long().to(device),
              x['static_numeric'].float().to(device),
              x['ts_cat'].long().to(device),
              x['ts_numeric'].float().to(device)
             )
        loss = criterion(out, x['y'].float().to(device))
        losses.append(loss.item())
        if len(losses) == 100:
            print(torch.mean(torch.tensor(losses)))
            losses = []
        loss.backward()
        optimizer.step() 
        optimizer.zero_grad()
    
    model.eval()
    outs, ys = [], []
    with torch.no_grad():
        for x in valid_dl:
            out = model(x["static_cat"].long().to(device),
              x['static_numeric'].float().to(device),
              x['ts_cat'].long().to(device),
              x['ts_numeric'].float().to(device)
            )
            ys.append(x['y'].numpy())
            outs.append(out.detach().cpu().numpy())
            
    outs = np.concatenate(outs, axis=-1)
    ys = np.concatenate(ys, axis=-1)
    
    outs = target_scaler.inverse_transform(outs)
    ys = target_scaler.inverse_transform(ys)
    
    m = metric(outs, ys)
    if m > best_metric:
        torch.save(model.state_dict(), 'model0.pth')
        best_metric = m
    model.train()
    optimizer.zero_grad()